In [ ]:
# 실전 예제로 주식 데이터(삼성전자, 카카오, 네이버) 를 활용하여
# 주가 변동을 시각화하는 예제를 만들어보겠습니다.
#
# 이 예제에서는 Matplotlib과 Pandas를 활용하여 다음을 분석합니다:
#
# 주가 변동 라인 그래프
# 삼성전자, 카카오, 네이버의 주가 변동을 하나의 그래프로 시각화합니다.
#
# 이동 평균선 추가
# 단기(5일) 및 장기(20일) 이동 평균선을 추가하여 주가 추세를 분석합니다.
#
# 히트맵을 통한 상관관계 분석
# 삼성전자, 카카오, 네이버의 주가 상관관계를 히트맵으로 표현합니다.
#
#
# 1. 주식 데이터 수집 및 처리
# 먼저, 삼성전자(005930), 카카오(035720), 네이버(035420)의 주가 데이터를 가져오겠습니다.
# 실시간 주식 데이터를 사용하기 위해, web을 이용하여 최신 데이터를 검색합니다.
#
# 실전 예제로 삼성전자, 네이버, 카카오의 주가 데이터를 활용하여 주가 변동을 시각화하는 방법을 안내하겠습니다. 이를 통해 주가 변동 추이와 종목 간 상관관계를 분석할 수 있습니다.
#
#
#
# 1. 주가 데이터 수집
# 주가 데이터를 수집하기 위해 yahoo_fin 라이브러리를 사용합니다. 이 라이브러리는 야후 파이낸스에서 주가 데이터를 가져올 수 있도록 도와줍니다.
#
# 필요한 라이브러리 설치:
# !pip install yahoo_fin pandas matplotlib seaborn

In [1]:
!pip install yahoo_fin pandas matplotlib seaborn
!pip install requests_html

Defaulting to user installation because normal site-packages is not writeable


In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from yahoo_fin import stock_info as si

# 종목 코드 설정
tickers = {
    '삼성전자': '005930.KS',
    '네이버': '035420.KS',
    '카카오': '035720.KS'
}

# 데이터 수집
data = {}
for company, ticker in tickers.items():
    data[company] = si.get_data(ticker, start_date='2022-01-01', end_date='2025-01-01')

JSONDecodeError: Expecting value: line 1 column 1 (char 0)